# OpenFF NSP Optimization Set 2 Nitrogen v4.0

This notebook generates optimizations for phosphorus set of molecules in the NSP molecules set obtained from Pubchem.

In [1]:
import pathlib
from pprint import pprint
from openff.toolkit import Molecule
import numpy as np

from openff.qcsubmit import workflow_components
from openff.qcsubmit.factories import OptimizationDatasetFactory
from openff.qcsubmit.utils.visualize import molecules_to_pdf


In [2]:
input_file = 'set2-N-smiles-pka-normalized.smi'
molecules = Molecule.from_file(input_file, allow_undefined_stereo=True)
molecules_to_pdf(molecules, f"dataset.pdf")

In [3]:
len(molecules)

1178

In [4]:
dataset_factory = OptimizationDatasetFactory()
dataset_factory.add_workflow_components(
    workflow_components.StandardConformerGenerator(max_conformers=10)
)

description = """\
Optimization dataset to probe coverage of nitrogen, sulfur, and phosphorus (i.e., NSP) functional groups in general, and nitrogen in particular. Molecules were curated from PubChem datasets, retaining those with HAC < 40 and applying additional filters; the preprocessing steps are detailed at https://github.com/pavankum/NSP_sets. This dataset includes a broader set of molecules, many of which may not be drug-like, but are informative for differentiating force field parameter ranges. In addition to the default OpenFF QC specification (B3LYP-D3BJ/DZVP), an SPICE-level QC specification (ωB97M-D3BJ/def2-TZVPPD) is included, since many molecules have charged states. Set2 prioritizes as many varied charged states as possible with valid valences for the atoms. Some of these may include charged states that get protonated immediately in solvent, but they were retained for experiments to check the quality of charge equilibration in MLIPs as some of these may be used as reference for bespoke torsion fits. The set of smiles were pka-normalized using Openeye filter.

- Number of unique molecules: 1166
- Number of filtered molecules: 1
- Number of conformers: 7421
- Number of conformers per molecule (min, mean, max): 1, 6.36, 10
- Mean molecular weight: 282.17
- Min molecular weight: 67.09
- Max molecular weight: 592.90
- Charges: [-10.0, -8.0, -7.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
## Metadata
- Elements: {O, C, Cl, N, S, P, H, Br, I, F}
- Spec: default
	 - basis: DZVP
	 - implicit_solvent: None
	 - keywords: {}
	 - maxiter: 200
	 - method: B3LYP-D3BJ
	 - program: psi4
	- SCF properties:
		- dipole
		- quadrupole
		- wiberg_lowdin_indices
		- mayer_indices
		- lowdin_charges
		- mulliken_charges
- Spec: WB97M-D3BJ/def2-TZVPPD
	 - basis: def2-TZVPPD
	 - implicit_solvent: None
	 - keywords: {}
	 - maxiter: 200
	 - method: WB97M-D3BJ
	 - program: psi4
	- SCF properties:
		- dipole
		- quadrupole
		- wiberg_lowdin_indices
		- mayer_indices
		- lowdin_charges
		- mulliken_charges
"""

dataset = dataset_factory.create_dataset(
    dataset_name="OpenFF NSP Optimization Set 2 Nitrogen v4.0",
    tagline="Molecules curated from PubChem for Nitrogen",
    description=description,
    molecules=molecules,
)

dataset.metadata.submitter = "pavankum"
dataset.metadata.long_description_url = (
    "https://github.com/openforcefield/qca-dataset-submission/tree/master/"
    "submissions/" + str(pathlib.Path.cwd().name)
)

Preparation                   : 100%|███████| 1166/1166 [02:47<00:00,  6.96it/s]


In [5]:
dataset.n_molecules

1166

Add a new qcspecification to the factory which will be applied to the dataset.
    
    Parameters:
        method: The name of the method to use eg B3LYP-D3BJ
        basis: The name of the basis to use can also be `None`
        program: The name of the program to execute the computation
        spec_name: The name the spec should be stored under
        spec_description: The description of the spec
        store_wavefunction: what parts of the wavefunction that should be saved
        overwrite: If there is a spec under this name already overwrite it
        implicit_solvent: The implicit solvent settings if it is to be used.
        maxiter: The maximum number of SCF iterations that should be done.
        scf_properties: The list of SCF properties that should be extracted from the calculation.
        keywords: Program specific computational keywords that should be passed to
            the program

In [6]:


dataset.add_qc_spec(method='WB97M-D3BJ',
                    basis='def2-TZVPPD',
                    program='psi4',
                    spec_name="WB97M-D3BJ/def2-TZVPPD",
                    spec_description="SPICE sets level of theory",
                    store_wavefunction='none',
                    implicit_solvent=None,
                    maxiter=200,
                    scf_properties=['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices', 'lowdin_charges', 'mulliken_charges'],
                    keywords={}
                    )
dataset.add_qc_spec(method='B3LYP-D3BJ',
                    basis='DZVP',
                    program='psi4',
                    spec_name="default",
                    spec_description="Standard OpenFF optimization quantum chemistry specification.",
                    store_wavefunction='none',
                    implicit_solvent=None,
                    maxiter=200,
                    scf_properties=['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices', 'lowdin_charges', 'mulliken_charges'],
                    keywords={},
                    overwrite=True
                    )        


        

In [7]:
pprint(dataset.dict()['qc_specifications'])

{'WB97M-D3BJ/def2-TZVPPD': {'basis': 'def2-TZVPPD',
                            'implicit_solvent': None,
                            'keywords': {},
                            'maxiter': 200,
                            'method': 'WB97M-D3BJ',
                            'program': 'psi4',
                            'scf_properties': ['dipole',
                                               'quadrupole',
                                               'wiberg_lowdin_indices',
                                               'mayer_indices',
                                               'lowdin_charges',
                                               'mulliken_charges'],
                            'spec_description': 'SPICE sets level of theory',
                            'spec_name': 'WB97M-D3BJ/def2-TZVPPD',
                            'store_wavefunction': 'none'},
 'default': {'basis': 'DZVP',
             'implicit_solvent': None,
             'keywords': {},
             'maxi

In [8]:
# summarize dataset for readme
confs = np.array([len(mol.conformers) for mol in dataset.molecules])

print("- Number of unique molecules:", dataset.n_molecules)
print("- Number of filtered molecules:", dataset.n_filtered)
print("- Number of conformers:", sum(confs))
print(
    "- Number of conformers per molecule (min, mean, max): "
    f"{confs.min()}, {confs.mean():.2f}, {confs.max()}"
)

masses = [
    [
        sum([atom.mass.m for atom in molecule.atoms])
        for molecule in dataset.molecules
    ]
]
print(f"- Mean molecular weight: {np.mean(np.array(masses)):.2f}")
print(f"- Min molecular weight: {np.min(np.array(masses)):.2f}")
print(f"- Max molecular weight: {np.max(np.array(masses)):.2f}")
print("- Charges:", sorted(set(m.total_charge.m for m in dataset.molecules)))


print("## Metadata")
print(f"- Elements: {{{', '.join(dataset.metadata.dict()['elements'])}}}")


fields = [
    "basis",
    "implicit_solvent",
    "keywords",
    "maxiter",
    "method",
    "program",
]
for spec, obj in dataset.qc_specifications.items():
    od = obj.dict()
    print("- Spec:", spec)
    for field in fields:
        print(f"\t - {field}: {od[field]}")
    print("\t- SCF properties:")
    for field in od["scf_properties"]:
        print(f"\t\t- {field}")


# export the dataset
dataset.export_dataset("dataset.json.bz2")
dataset.molecules_to_file("dataset.smi", "smi")
dataset.visualize("dataset.pdf", columns=4)

- Number of unique molecules: 1166
- Number of filtered molecules: 1
- Number of conformers: 7421
- Number of conformers per molecule (min, mean, max): 1, 6.36, 10
- Mean molecular weight: 282.17
- Min molecular weight: 67.09
- Max molecular weight: 592.90
- Charges: [-10.0, -8.0, -7.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
## Metadata
- Elements: {O, Cl, H, P, C, Br, N, I, S, F}
- Spec: default
	 - basis: DZVP
	 - implicit_solvent: None
	 - keywords: {}
	 - maxiter: 200
	 - method: B3LYP-D3BJ
	 - program: psi4
	- SCF properties:
		- dipole
		- quadrupole
		- wiberg_lowdin_indices
		- mayer_indices
		- lowdin_charges
		- mulliken_charges
- Spec: WB97M-D3BJ/def2-TZVPPD
	 - basis: def2-TZVPPD
	 - implicit_solvent: None
	 - keywords: {}
	 - maxiter: 200
	 - method: WB97M-D3BJ
	 - program: psi4
	- SCF properties:
		- dipole
		- quadrupole
		- wiberg_lowdin_indices
		- mayer_indices
		- lowdin_charges
		- mulliken_charges
